In [1]:
import numpy as np
from random import shuffle
from math import log, floor
import pandas as pd
import tensorflow as tf
import tensorboard as tb
from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
from collections import Counter
from keras import *
from keras.engine.topology import *
from keras.optimizers import *
import keras
# import pandas as pd
import glob
from sklearn.semi_supervised import *
import pickle
from keras.applications import *
from keras.preprocessing.image import *
from keras.losses import mse, binary_crossentropy
import pandas as pd # data frame
import numpy as np # matrix math
from scipy.io import wavfile # reading the wavfile
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
import matplotlib.pyplot as plt # to view graphs
import wave
from math import log, floor
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import numpy as np
import pandas as pd
from sklearn.decomposition import *
from sklearn.cluster import KMeans
import sys, os
import random,math
from tqdm import tqdm ##
from xgboost.sklearn import XGBClassifier
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
# import xgboost as xgb
# import lightgbm as lgb
# import catboost as ctb
from keras.utils import *
from sklearn.ensemble import *
import pickle
# from bayes_opt import BayesianOptimization
from logHandler import Logger
from utils import readCSV, getPath, writePickle,readPickle
from keras.regularizers import l2
from keras.callbacks import History ,ModelCheckpoint, EarlyStopping
from keras.utils.generic_utils import get_custom_objects

import resnet
from random_eraser import get_random_eraser
from mixup_generator import MixupGenerator

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# original stacking 41_dim + normalize + softmax

take train_X

In [2]:
# type_ = 'mfcc7' #要抽取哪一個種類的unverified trainX出來去re-train
un_or_test = 'combine' # unverified or test
phase = 'stacking'

folder = 'data/'+phase+'/'+un_or_test+'/' #共同predict對unverified data的結果


acc_df = pd.read_csv('data/'+phase+'/weight_acc4.csv') # acc csv檔名格式: (csv,acc)
# acc_df.columns = ['unverified','test','acc']
acc_df.columns = [un_or_test,'acc']
acc_df = acc_df.filter([un_or_test,'acc'])
files = os.listdir(folder)

ratio_all=0
for i,csv in enumerate(files):
    if csv.startswith('valid_acc'):
        continue
    else:
        ratio = acc_df[acc_df[un_or_test] == csv]['acc'].values[0]
#         print(ratio)
        ratio_all += ratio
    df = pd.read_csv(os.path.join(folder,csv)) #ori method
#     df = pd.read_csv(os.path.join(folder,csv),header=0,index_col=0) # new method
    df.sort_values("fname", inplace=True)
    if df.iloc[0,0] == 'fname':
        df = df.drop(0,axis=0)
#     df = df.drop(0,axis=1) #ori method
    df = df.drop(['fname'],axis=1) #mew mthod

    if i==0:
        train_X = df.values*ratio
    else:
#         try:
        train_X += df.values*ratio 
#         except:
#             train_X += df.values[5763:]*ratio
print(train_X.shape)
train_X = train_X / ratio_all
print(sum(train_X[0]))
reverse_dict = pickle.load(open('data/map_reverse.pkl' , 'rb'))

(3710, 41)
1.0000000108918383


take train_Y

In [3]:
label = pd.read_csv('data/train_label.csv',names=['fname','label','verified'],header=0)
label = label[label['verified']==1]
dicts_ = pickle.load(open('data/map.pkl','rb'))
label['trans']=label['label'].map(dicts_)
label = label.drop(['verified','label'],axis=1)
# label = pd.merge(label,df_dict[k],on='fname',how='inner')
label.sort_values("fname", inplace=True)
label = label['trans'].values
print('label like:',label,'data#:', len(label))

label like: [ 1  3  4 ... 14  9 17] data#: 3710


Split Data

In [74]:
def split_valid_set(X_all, Y_all, percentage):
    all_data_size = len(X_all)
    valid_data_size = int(floor(all_data_size * percentage))

    X_all, Y_all = _shuffle(X_all, Y_all)

    X_train, Y_train = X_all[0:valid_data_size], Y_all[0:valid_data_size]
    X_valid, Y_valid = X_all[valid_data_size:], Y_all[valid_data_size:]

    return X_train, Y_train, X_valid, Y_valid

In [75]:
def _shuffle(X, Y):
    randomize = np.arange(len(X))
    np.random.shuffle(randomize)
#     print(X.shape, Y.shape)
    return (X[randomize], Y[randomize])

In [76]:
kk = np.argmax(train_X,axis=1)
count=0
for i,v in enumerate(kk):
    if v == label[i]:
        count+=1
print('ens acc', count/len(kk))

ens acc 0.8264150943396227


In [77]:

label = to_categorical(label,num_classes=41)
X_train, Y_train, X_valid, Y_valid = split_valid_set(train_X, label, 0.95)
print(X_train.shape , Y_train.shape)

(3524, 41) (3524, 41)


## Model

In [2]:
class ReLUs(Activation):
    def __init__(self, activation, **kwargs):
        super(ReLUs, self).__init__(activation, **kwargs)
        self.__name__ = 'swish'

def swish(x):
    # Your activation function specialties here
    A = (K.sigmoid(x) * x)
    return A


get_custom_objects().update({'swish': ReLUs(swish)})

In [107]:

input_ = Input(shape=(41,))
# bn = BatchNormalization()(input_)
dense = Dense(41,activation='selu',kernel_initializer='lecun_normal')(input_) #swish,he_uniform
# dense = Dense(8,kernel_initializer='uniform',kernel_regularizer=l2(0.000008))(bn)
bn = BatchNormalization()(dense)
# dense = LeakyReLU()(dense)

dropout1 = Dropout(0.0005)(bn)

# dense = Dense(164,kernel_initializer='uniform',kernel_regularizer=l2(0.008))(dropout)
# bn = BatchNormalization()(dropout)
for i in range(4):
    dropout = Concatenate()([dropout1,input_])
    dense = Dense(82,activation='selu',kernel_initializer='lecun_normal')(dropout)
    # dense = Dense(41,kernel_initializer='uniform',kernel_regularizer=l2(0.000008))(bn)
    bn = BatchNormalization()(dense)
    # dense = LeakyReLU()(dense)
    dropout2 = Dropout(0.5005)(bn)
    
    dropout = Concatenate()([dropout2,dropout1])
    dense = Dense(82,activation='selu',kernel_initializer='lecun_normal')(dropout)
    bn = BatchNormalization()(dense)
    dropout1 = Dropout(0.5005)(bn)

# bn = BatchNormalization()(dropout)
dense = Dense(41,activation='softmax',kernel_initializer='uniform')(dropout1)
# dense = Dense(41,activation='softmax',kernel_initializer='uniform',kernel_regularizer=l2(0.000008))(dropout)
# dense = LeakyReLU()(dense)
model = Model(inputs=input_, outputs=dense)
# model = Sequential()
# i
# model.add(BatchNormalization())
# model.add(Dense(41,activation='linear',input_shape=(41,)))
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_24 (InputLayer)           (None, 41)           0                                            
__________________________________________________________________________________________________
dense_195 (Dense)               (None, 41)           1722        input_24[0][0]                   
__________________________________________________________________________________________________
batch_normalization_174 (BatchN (None, 41)           164         dense_195[0][0]                  
__________________________________________________________________________________________________
dropout_173 (Dropout)           (None, 41)           0           batch_normalization_174[0][0]    
__________________________________________________________________________________________________
concatenat

In [108]:
batchSize = 128
patien=100
epoch=3000
feature_type = 'stacking'
saveD = 'model/'+feature_type+'/'
opt = Nadam()#Adam(decay=1e-20)#Nadam() #Adam(lr=2e-3,decay=1e-20)
if not os.path.exists(saveD):
    os.makedirs(saveD)


model.compile(loss=['categorical_crossentropy'],optimizer=opt, metrics=['acc']) 
logD = './logs/'+feature_type+'/'
history = History()
callback=[
    ReduceLROnPlateau(monitor='loss', factor=0.5, patience=int(patien/10),min_lr=1e-6,
                      mode='min', cooldown=1,verbose=1 ), #0.2,/25 #0.5/5/3e-6 #0.3/10/1e-6
    EarlyStopping(patience=patien,monitor='val_loss',verbose=1,
                  mode='min'),
    ModelCheckpoint(saveD+'LGD_fold_stacking_41_only1'+'.h5',
                    monitor='val_acc',verbose=1,save_best_only=True, 
                    save_weights_only=False,
                    mode='max'),
    TensorBoard(log_dir=logD+'LGD_fold_stacking_41'),
    history
]


model.fit(X_train,Y_train,
          shuffle=True,
          callbacks=callback, 
          class_weight='auto',
          validation_data=(X_valid,Y_valid),
          batch_size=batchSize,
          epochs=epoch)


#0.87097 #model/stacking/LGD_fold_stacking_ori_resnet.h5 (X)
#0.82258  model/stacking/LGD_fold_stacking_41_densenet.h5
#0.82796  model/stacking/LGD_fold_stacking_41_densenet2.h5
#0.85484 model/stacking/LGD_fold_stacking_41_only1.h5

Train on 3524 samples, validate on 186 samples
Epoch 1/3000
3524/3524 [==============================] - 2s 617us/step - loss: 3.6016 - acc: 0.0840 - val_loss: 2.9878 - val_acc: 0.3172

Epoch 00001: val_acc improved from -inf to 0.31720, saving model to model/stacking/LGD_fold_stacking_41_only1.h5
Epoch 2/3000
3524/3524 [==============================] - 0s 81us/step - loss: 3.1018 - acc: 0.2094 - val_loss: 2.6199 - val_acc: 0.3763

Epoch 00002: val_acc improved from 0.31720 to 0.37634, saving model to model/stacking/LGD_fold_stacking_41_only1.h5
Epoch 3/3000
3524/3524 [==============================] - 0s 87us/step - loss: 2.7073 - acc: 0.2963 - val_loss: 2.1602 - val_acc: 0.4946

Epoch 00003: val_acc improved from 0.37634 to 0.49462, saving model to model/stacking/LGD_fold_stacking_41_only1.h5
Epoch 4/3000
3524/3524 [==============================] - 0s 80us/step - loss: 2.4580 - acc: 0.3556 - val_loss: 1.9123 - val_acc: 0.5161

Epoch 00004: val_acc improved from 0.49462 to 0.51613, 

Epoch 38/3000
3524/3524 [==============================] - 0s 97us/step - loss: 0.8835 - acc: 0.7747 - val_loss: 0.8265 - val_acc: 0.8280

Epoch 00038: val_acc did not improve from 0.83333
Epoch 39/3000
3524/3524 [==============================] - 0s 84us/step - loss: 0.8395 - acc: 0.7809 - val_loss: 0.7960 - val_acc: 0.8333

Epoch 00039: val_acc did not improve from 0.83333
Epoch 40/3000
3524/3524 [==============================] - 0s 80us/step - loss: 0.8292 - acc: 0.7846 - val_loss: 0.7603 - val_acc: 0.8548

Epoch 00040: val_acc improved from 0.83333 to 0.85484, saving model to model/stacking/LGD_fold_stacking_41_only1.h5
Epoch 41/3000
3524/3524 [==============================] - 0s 80us/step - loss: 0.8168 - acc: 0.7894 - val_loss: 0.7706 - val_acc: 0.8441

Epoch 00041: val_acc did not improve from 0.85484
Epoch 42/3000
3524/3524 [==============================] - 0s 80us/step - loss: 0.8295 - acc: 0.7886 - val_loss: 0.7819 - val_acc: 0.8441

Epoch 00042: val_acc did not improve fr

3524/3524 [==============================] - 0s 86us/step - loss: 0.6499 - acc: 0.8238 - val_loss: 0.7106 - val_acc: 0.8333

Epoch 00081: val_acc did not improve from 0.85484
Epoch 82/3000
3524/3524 [==============================] - 0s 84us/step - loss: 0.6214 - acc: 0.8306 - val_loss: 0.7142 - val_acc: 0.8333

Epoch 00082: val_acc did not improve from 0.85484
Epoch 83/3000
3524/3524 [==============================] - 0s 83us/step - loss: 0.6158 - acc: 0.8275 - val_loss: 0.7144 - val_acc: 0.8280

Epoch 00083: val_acc did not improve from 0.85484
Epoch 84/3000
3524/3524 [==============================] - 0s 80us/step - loss: 0.6350 - acc: 0.8201 - val_loss: 0.7097 - val_acc: 0.8333

Epoch 00084: val_acc did not improve from 0.85484
Epoch 85/3000
3524/3524 [==============================] - 0s 81us/step - loss: 0.6149 - acc: 0.8258 - val_loss: 0.7025 - val_acc: 0.8333

Epoch 00085: val_acc did not improve from 0.85484
Epoch 86/3000
3524/3524 [==============================] - 0s 83us/st

3524/3524 [==============================] - 0s 93us/step - loss: 0.5407 - acc: 0.8482 - val_loss: 0.7117 - val_acc: 0.8280

Epoch 00123: val_acc did not improve from 0.85484
Epoch 124/3000
3524/3524 [==============================] - 0s 98us/step - loss: 0.5676 - acc: 0.8482 - val_loss: 0.7149 - val_acc: 0.8280

Epoch 00124: val_acc did not improve from 0.85484
Epoch 125/3000
3524/3524 [==============================] - 0s 100us/step - loss: 0.5479 - acc: 0.8505 - val_loss: 0.7188 - val_acc: 0.8280

Epoch 00125: val_acc did not improve from 0.85484
Epoch 126/3000
3524/3524 [==============================] - 0s 108us/step - loss: 0.5472 - acc: 0.8465 - val_loss: 0.7212 - val_acc: 0.8280

Epoch 00126: val_acc did not improve from 0.85484
Epoch 127/3000
3524/3524 [==============================] - 0s 104us/step - loss: 0.5443 - acc: 0.8459 - val_loss: 0.7214 - val_acc: 0.8280

Epoch 00127: val_acc did not improve from 0.85484
Epoch 128/3000
3524/3524 [==============================] - 0s

3524/3524 [==============================] - 0s 83us/step - loss: 0.5379 - acc: 0.8547 - val_loss: 0.7215 - val_acc: 0.8280

Epoch 00165: val_acc did not improve from 0.85484
Epoch 166/3000
3524/3524 [==============================] - 0s 88us/step - loss: 0.5485 - acc: 0.8499 - val_loss: 0.7203 - val_acc: 0.8226

Epoch 00166: val_acc did not improve from 0.85484
Epoch 167/3000
3524/3524 [==============================] - 0s 84us/step - loss: 0.5514 - acc: 0.8417 - val_loss: 0.7199 - val_acc: 0.8226

Epoch 00167: val_acc did not improve from 0.85484
Epoch 168/3000
3524/3524 [==============================] - 0s 82us/step - loss: 0.5375 - acc: 0.8527 - val_loss: 0.7197 - val_acc: 0.8226

Epoch 00168: val_acc did not improve from 0.85484
Epoch 169/3000
3524/3524 [==============================] - 0s 86us/step - loss: 0.5326 - acc: 0.8536 - val_loss: 0.7195 - val_acc: 0.8226

Epoch 00169: val_acc did not improve from 0.85484
Epoch 170/3000
3524/3524 [==============================] - 0s 86

# 410-dim (model-self ensemble)+sigmoid

get train X

In [2]:
# type_ = 'mfcc7' #要抽取哪一個種類的unverified trainX出來去re-train
un_or_test = 'combine' # unverified or test
phase = 'stacking'

folder = 'data/'+phase+'/'+un_or_test+'/' #共同predict對unverified data的結果


acc_df = pd.read_csv('data/'+phase+'/weight_acc4.csv') # acc csv檔名格式: (csv,acc)
# acc_df.columns = ['unverified','test','acc']
acc_df.columns = [un_or_test,'acc']
acc_df = acc_df.filter([un_or_test,'acc'])
files = os.listdir(folder)

ratio_all=0
for i,csv in enumerate(files):
    if csv.startswith('valid_acc'):
        continue
    else:
        ratio = acc_df[acc_df[un_or_test] == csv]['acc'].values[0]
#         print(ratio)
        ratio_all += ratio
    df = pd.read_csv(os.path.join(folder,csv)) #ori method
#     df = pd.read_csv(os.path.join(folder,csv),header=0,index_col=0) # new method
    df.sort_values("fname", inplace=True)
    if df.iloc[0,0] == 'fname':
        df = df.drop(0,axis=0)
#     df = df.drop(0,axis=1) #ori method
    df = df.drop(['fname'],axis=1) #mew mthod

    if i==0:
        train_X = df.values*ratio
    else:
        train_X_ = df.values*ratio 
        train_X = np.concatenate((train_X,train_X_),axis=1)
print(train_X.shape)
# train_X = train_X / ratio_all
print(sum(train_X[0]))
reverse_dict = pickle.load(open('data/map_reverse.pkl' , 'rb'))

(3710, 410)
86.97535094732149


take trainY

In [3]:
label = pd.read_csv('data/train_label.csv',names=['fname','label','verified'],header=0)
label = label[label['verified']==1]
dicts_ = pickle.load(open('data/map.pkl','rb'))
label['trans']=label['label'].map(dicts_)
label = label.drop(['verified','label'],axis=1)
# label = pd.merge(label,df_dict[k],on='fname',how='inner')
label.sort_values("fname", inplace=True)
label = label['trans'].values
print('label like:',label,'data#:', len(label))

label like: [ 1  3  4 ... 14  9 17] data#: 3710


split data

In [31]:
def split_valid_set(X_all, Y_all, percentage):
    all_data_size = len(X_all)
    valid_data_size = int(floor(all_data_size * percentage))

    X_all, Y_all = _shuffle(X_all, Y_all)

    X_train, Y_train = X_all[0:valid_data_size], Y_all[0:valid_data_size]
    X_valid, Y_valid = X_all[valid_data_size:], Y_all[valid_data_size:]

    return X_train, Y_train, X_valid, Y_valid

In [32]:
def _shuffle(X, Y):
    randomize = np.arange(len(X))
    np.random.shuffle(randomize)
#     print(X.shape, Y.shape)
    return (X[randomize], Y[randomize])

In [33]:

label = to_categorical(label,num_classes=41)
X_train, Y_train, X_valid, Y_valid = split_valid_set(train_X, label, 0.95)
print(X_train.shape , Y_train.shape)

(3524, 410) (3524, 41)


## Model

In [40]:
input_ = Input(shape=(410,))
do1 = input_
# for i in range(101):
#     add = Concatenate()([input_,do1])
#     dense = Dense(820,activation='selu',kernel_initializer='lecun_normal')(add)
#     bn = BatchNormalization()(dense)
#     do2 = Dropout(0.72)(bn)
#     add = Concatenate()([do2,do1])
#     dense = Dense(820,activation='selu',kernel_initializer='lecun_normal')(add)
#     bn = BatchNormalization()(dense)
#     do1 = Dropout(0.72)(bn)

# dense = Dense(512,activation='swish',kernel_initializer='he_uniform')(input_)
# bn = BatchNormalization()(dense)
# do = Dropout(0.2)(bn)
# dense = Dense(512,activation='swish',kernel_initializer='he_uniform')(do)
# bn = BatchNormalization()(dense)
# do = Dropout(0.2)(bn)

# dense = Dense(64,activation='swish',kernel_initializer='he_uniform')(do)
# bn = BatchNormalization()(dense)
# do = Dropout(0.36)(bn)
dense = Dense(41,activation='swish',kernel_initializer='he_uniform')(do1)
bn = BatchNormalization()(dense)
do = Dropout(0.00001)(bn)

dense = Dense(41,activation='sigmoid',kernel_initializer='uniform')(do)
# dense = Dense(41,activation='softmax',kernel_initializer='uniform',kernel_regularizer=l2(0.000008))(dropout)
# dense = LeakyReLU()(dense)
model = Model(inputs=input_, outputs=dense)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 410)               0         
_________________________________________________________________
dense_372 (Dense)            (None, 41)                16851     
_________________________________________________________________
batch_normalization_363 (Bat (None, 41)                164       
_________________________________________________________________
dropout_363 (Dropout)        (None, 41)                0         
_________________________________________________________________
dense_373 (Dense)            (None, 41)                1722      
Total params: 18,737
Trainable params: 18,655
Non-trainable params: 82
_________________________________________________________________


In [42]:
batchSize = 256
patien=100
epoch=3000
feature_type = 'stacking'
saveD = 'model/'+feature_type+'/'
opt = Adam(decay=1e-20)#Nadam() #Adam(lr=2e-3,decay=1e-20)
if not os.path.exists(saveD):
    os.makedirs(saveD)


model.compile(loss=['categorical_crossentropy'],optimizer=opt, metrics=['acc']) 
logD = './logs/'+feature_type+'/'
history = History()
callback=[
    ReduceLROnPlateau(monitor='loss', factor=0.5, patience=int(patien/10),min_lr=1e-6,
                      mode='min', cooldown=1,verbose=1 ), #0.2,/25 #0.5/5/3e-6 #0.3/10/1e-6
    EarlyStopping(patience=patien,monitor='val_loss',verbose=1,
                  mode='min'),
    ModelCheckpoint(saveD+'LGD_fold_stacking_410_only3'+'.h5',
                    monitor='val_acc',verbose=1,save_best_only=True, 
                    save_weights_only=False,
                    mode='max'),
    TensorBoard(log_dir=logD+'LGD_fold_stacking_410_resnet'),
    history
]


model.fit(X_train,Y_train,
          shuffle=True,
          callbacks=callback, 
          class_weight='auto',
          validation_data=(X_valid,Y_valid),
          batch_size=batchSize,
          epochs=epoch)


#0.90860=>model/stacking/LGD_fold_stacking_410_resnet.h5
# 0.91935=>model/stacking/LGD_fold_stacking_410_resnet2.h5
#0.92473=> model/stacking/LGD_fold_stacking_410_dense1.h5
#0.92473=> model/stacking/LGD_fold_stacking_410_dense2.h5
# 0.9355=>model/stacking/LGD_fold_stacking_410_dense3.h5
#0.93011=> model/stacking/LGD_fold_stacking_410_dense4.h5
#0.93011=>model/stacking/LGD_fold_stacking_410_only1.h5
#0.95699=>model/stacking/LGD_fold_stacking_410_only2.h5


Train on 3524 samples, validate on 186 samples
Epoch 1/3000
3524/3524 [==============================] - 2s 429us/step - loss: 0.0031 - acc: 0.9994 - val_loss: 0.7190 - val_acc: 0.8817

Epoch 00001: val_acc improved from -inf to 0.88172, saving model to model/stacking/LGD_fold_stacking_410_only3.h5
Epoch 2/3000
3524/3524 [==============================] - 0s 20us/step - loss: 0.0023 - acc: 0.9994 - val_loss: 0.7335 - val_acc: 0.8817

Epoch 00002: val_acc did not improve from 0.88172
Epoch 3/3000
3524/3524 [==============================] - 0s 20us/step - loss: 0.0018 - acc: 0.9994 - val_loss: 0.7436 - val_acc: 0.8763

Epoch 00003: val_acc did not improve from 0.88172
Epoch 4/3000
3524/3524 [==============================] - 0s 19us/step - loss: 0.0015 - acc: 0.9997 - val_loss: 0.7548 - val_acc: 0.8817

Epoch 00004: val_acc did not improve from 0.88172
Epoch 5/3000
3524/3524 [==============================] - 0s 19us/step - loss: 0.0013 - acc: 0.9997 - val_loss: 0.7605 - val_acc: 0.8817


Epoch 00042: val_acc did not improve from 0.88172
Epoch 43/3000
3524/3524 [==============================] - 0s 20us/step - loss: 6.3929e-05 - acc: 1.0000 - val_loss: 0.9364 - val_acc: 0.8710

Epoch 00043: val_acc did not improve from 0.88172
Epoch 44/3000
3524/3524 [==============================] - 0s 20us/step - loss: 6.3968e-05 - acc: 1.0000 - val_loss: 0.9358 - val_acc: 0.8710

Epoch 00044: val_acc did not improve from 0.88172
Epoch 45/3000
3524/3524 [==============================] - 0s 20us/step - loss: 6.2027e-05 - acc: 1.0000 - val_loss: 0.9382 - val_acc: 0.8710

Epoch 00045: val_acc did not improve from 0.88172
Epoch 46/3000
3524/3524 [==============================] - 0s 20us/step - loss: 6.1107e-05 - acc: 1.0000 - val_loss: 0.9396 - val_acc: 0.8710

Epoch 00046: val_acc did not improve from 0.88172
Epoch 47/3000
3524/3524 [==============================] - 0s 20us/step - loss: 5.9465e-05 - acc: 1.0000 - val_loss: 0.9414 - val_acc: 0.8710

Epoch 00047: val_acc did not impro

3524/3524 [==============================] - 0s 23us/step - loss: 4.6524e-05 - acc: 1.0000 - val_loss: 0.9555 - val_acc: 0.8710

Epoch 00083: val_acc did not improve from 0.88172
Epoch 84/3000
3524/3524 [==============================] - 0s 20us/step - loss: 4.7041e-05 - acc: 1.0000 - val_loss: 0.9556 - val_acc: 0.8710

Epoch 00084: val_acc did not improve from 0.88172
Epoch 85/3000
3524/3524 [==============================] - 0s 20us/step - loss: 4.6152e-05 - acc: 1.0000 - val_loss: 0.9557 - val_acc: 0.8710

Epoch 00085: val_acc did not improve from 0.88172
Epoch 86/3000
3524/3524 [==============================] - 0s 20us/step - loss: 4.7864e-05 - acc: 1.0000 - val_loss: 0.9557 - val_acc: 0.8710

Epoch 00086: val_acc did not improve from 0.88172
Epoch 87/3000
3524/3524 [==============================] - 0s 20us/step - loss: 4.7311e-05 - acc: 1.0000 - val_loss: 0.9558 - val_acc: 0.8710

Epoch 00087: val_acc did not improve from 0.88172
Epoch 88/3000
3524/3524 [========================

# Label Spreading

get testX

In [3]:
# 410-dim
un_or_test = 'combine_test' # unverified or test
phase = 'stacking'

folder = 'data/'+phase+'/'+un_or_test+'/' #共同predict對unverified data的結果


acc_df = pd.read_csv('data/'+phase+'/weight_acc4.csv') # acc csv檔名格式: (csv,acc)
# acc_df.columns = ['unverified','test','acc']
acc_df.columns = [un_or_test,'acc']
acc_df = acc_df.filter([un_or_test,'acc'])
files = os.listdir(folder)

ratio_all=0
for i,csv in enumerate(files):
    if csv.startswith('valid_acc'):
        continue
    else:
        ratio = acc_df[acc_df[un_or_test] == csv]['acc'].values[0]
#         print(ratio)
        ratio_all += ratio
    df = pd.read_csv(os.path.join(folder,csv)) #ori method
#     df = pd.read_csv(os.path.join(folder,csv),header=0,index_col=0) # new method
#     df.sort_values("fname", inplace=True)
    if df.iloc[0,0] == 'fname':
        df = df.drop(0,axis=0)
#     df = df.drop(0,axis=1) #ori method
    df = df.drop(['fname'],axis=1) #mew mthod
    df = df.drop(['Unnamed: 0'],axis=1)
    if i==0:
        test_X = df.values*ratio
    else:
        test_X_ = df.values*ratio 
        test_X = np.concatenate((test_X,test_X_),axis=1)
print(test_X.shape)
# test_X = test_X / ratio_all
print(sum(test_X[0]))
reverse_dict = pickle.load(open('data/map_reverse.pkl' , 'rb'))

#0.90860=>model/stacking/LGD_fold_stacking_410_resnet.h5
# 0.91935=>model/stacking/LGD_fold_stacking_410_resnet2.h5
#0.92473=> model/stacking/LGD_fold_stacking_410_dense1.h5
#0.92473=> model/stacking/LGD_fold_stacking_410_dense2.h5
# 0.9355=>model/stacking/LGD_fold_stacking_410_dense3.h5
#0.93011=> model/stacking/LGD_fold_stacking_410_dense4.h5
#0.93011=>model/stacking/LGD_fold_stacking_410_only1.h5
#0.95699=>model/stacking/LGD_fold_stacking_410_only2.h5

(9400, 410)
86.97534892253985


In [6]:
# 41-dim
un_or_test = 'combine_test' # unverified or test
phase = 'stacking'

folder = 'data/'+phase+'/'+un_or_test+'/' #共同predict對unverified data的結果


acc_df = pd.read_csv('data/'+phase+'/weight_acc4.csv') # acc csv檔名格式: (csv,acc)
# acc_df.columns = ['unverified','test','acc']
acc_df.columns = [un_or_test,'acc']
acc_df = acc_df.filter([un_or_test,'acc'])
files = os.listdir(folder)

ratio_all=0
for i,csv in enumerate(files):
    if csv.startswith('valid_acc'):
        continue
    else:
        ratio = acc_df[acc_df[un_or_test] == csv]['acc'].values[0]
#         print(ratio)
        ratio_all += ratio
    df = pd.read_csv(os.path.join(folder,csv)) #ori method
#     df = pd.read_csv(os.path.join(folder,csv),header=0,index_col=0) # new method
#     df.sort_values("fname", inplace=True)
    if df.iloc[0,0] == 'fname':
        df = df.drop(0,axis=0)
#     df = df.drop(0,axis=1) #ori method
    df = df.drop(['fname'],axis=1) #mew mthod
    df = df.drop(['Unnamed: 0'],axis=1)

    if i==0:
        test_X = df.values*ratio
    else:
#         try:
        test_X += df.values*ratio 
#         except:
#             test_X += df.values[5763:]*ratio
print(test_X.shape)
test_X = test_X / ratio_all
print(sum(test_X[0]))
reverse_dict = pickle.load(open('data/map_reverse.pkl' , 'rb'))
#0.87097 #model/stacking/LGD_fold_stacking_ori_resnet.h5 (X)
#0.82258  model/stacking/LGD_fold_stacking_41_densenet.h5
#0.82796  model/stacking/LGD_fold_stacking_41_densenet2.h5
#0.85484 model/stacking/LGD_fold_stacking_41_only1.h5

(9400, 41)
0.9999999876118902


In [11]:
name = 'LGD_fold_stacking_410_resnet'
model = load_model('model/stacking/'+name+'.h5') #change
# X_ver_dense = model.predict(train_X)
X_un_dense = model.predict(test_X)
print(X_un_dense.shape)
np.save('data/stacking/nn/'+name+'.npy',X_un_dense)

(9400, 41)


In [17]:
lp_model = [LabelSpreading(kernel='knn',n_jobs=-1,n_neighbors=int(3+6*random.random()),alpha=0.01+0.24*random.random(),max_iter=int(30+21*random.random()),tol = 0.0001+0.0015*random.random()) , LabelSpreading(kernel='rbf',n_jobs=-1, gamma=5+25*random.random(), max_iter=int(30+21*random.random()), alpha=0.01+0.24*random.random(),tol = 0.0001+0.0015*random.random())]
lp_model = random.choice(lp_model)
lp_model.fit(X_ver_dense,label)

LabelSpreading(alpha=0.19053594195851567, gamma=16.757711966287793,
        kernel='rbf', max_iter=30, n_jobs=-1, n_neighbors=7,
        tol=0.0006402983256596968)

In [18]:
test_ans = lp_model.predict(X_un_dense)
print(test_ans)
np.save('data/stacking/lp_model_res/Y_test_dense'+name+'.npy',test_ans)

[30 16 21 ...  4 21 23]


In [19]:
test_ans.shape

(9400,)

## Stacking Test ACC CSV

In [70]:
# type_ = 'mfcc7' #要抽取哪一個種類的unverified trainX出來去re-train
un_or_test = 'mow_resnet152_mfcc4' # unverified or test
phase = 'stacking/testing'

folder = 'data/'+phase+'/'+un_or_test+'/' #共同predict對unverified data的結果


acc_df = pd.read_csv('data/'+phase+'/valid_accF.csv') # acc csv檔名格式: (csv,acc)
# acc_df.columns = ['unverified','test','acc']
acc_df.columns = [un_or_test,'acc']
acc_df = acc_df.filter([un_or_test,'acc'])
files = os.listdir(folder)

ratio_all=0
for i,csv in enumerate(files):
    if csv.startswith('valid_acc'):
        continue
    else:
        ratio = acc_df[acc_df[un_or_test] == csv]['acc'].values[0]
#         print(ratio)
        ratio_all += ratio
    df = pd.read_csv(os.path.join(folder,csv)) #ori method
#     df = pd.read_csv(os.path.join(folder,csv),header=0,index_col=0) # new method
#     df.sort_values("fname", inplace=True)
    if df.iloc[0,0] == 'fname':
        df = df.drop(0,axis=0)
#     df = df.drop(0,axis=1) #ori method
    df = df.drop(['fname'],axis=1) #mew mthod

    if i==0:
        train_X = df.values*ratio
    else:
        try:
            train_X += df.values*ratio 
        except:
            train_X += df.values[5763:]*ratio
print(train_X.shape)
train_X = train_X / ratio_all
print(sum(train_X[0]))
reverse_dict = pickle.load(open('data/map_reverse.pkl' , 'rb'))

(9400, 41)
0.999999989562226


In [71]:
fname_test = pd.read_csv('data/sample_submission.csv')
fname_test =fname_test.filter(['fname'])
fname_test = pd.merge(fname_test,pd.DataFrame(train_X),how='inner',left_index=True,right_index=True)
fname_test.to_csv('data/'+phase+'/'+'mow_mfcc4_resnet152_phase3_val_all.csv',index=False)
fname_test

,fname,0,1,2,3,4,5,6,7,8,...,31,32,33,34,35,36,37,38,39,40
0,00063640.wav,6.716899e-03,0.003262,0.011294,6.656330e-03,6.437381e-03,3.693346e-03,6.623654e-03,5.046128e-03,1.009800e-01,...,3.721805e-03,4.707459e-02,1.368156e-03,0.000272,0.001281,9.495180e-04,3.004739e-03,0.006560,3.437896e-03,5.240033e-02
1,0013a1db.wav,1.356356e-02,0.018109,0.122530,3.021113e-03,2.173155e-02,5.141468e-03,1.631675e-03,4.415790e-02,5.313644e-03,...,2.063368e-03,1.419524e-02,1.114420e-02,0.007029,0.005043,3.751472e-03,1.808508e-03,0.007808,1.006945e-01,6.903243e-03
2,002bb878.wav,1.904144e-03,0.002737,0.004899,2.118041e-04,5.561231e-03,1.963949e-01,3.351156e-02,3.583244e-03,3.564368e-02,...,4.705913e-03,8.729406e-04,1.358890e-03,0.002554,0.001344,4.946434e-03,2.509053e-03,0.004161,3.681284e-03,1.738273e-03
3,002d392d.wav,6.030368e-03,0.002308,0.006066,1.652327e-03,6.434560e-03,7.567278e-03,4.879923e-03,2.862447e-03,2.699290e-03,...,2.332346e-03,4.569010e-03,3.937089e-03,0.011854,0.002886,1.868884e-03,1.511072e-03,0.003442,8.678284e-03,7.614618e-03
4,00326aa9.wav,9.933689e-04,0.003658,0.002760,3.949507e-04,9.805457e-04,7.272981e-04,5.191638e-04,6.691991e-03,1.259130e-04,...,6.034831e-04,3.013947e-04,5.089263e-04,0.001592,0.002250,3.023983e-04,3.128840e-04,0.000602,7.470297e-04,2.919013e-04
5,0038a046.wav,5.498995e-03,0.002947,0.006111,2.876998e-03,1.236199e-02,2.337747e-02,5.546759e-03,1.087584e-02,2.942085e-03,...,2.682927e-03,2.912839e-03,3.419625e-03,0.024355,0.003939,4.327347e-03,1.343031e-03,0.012234,1.185422e-02,6.204419e-03
6,003995fa.wav,5.915336e-03,0.005743,0.012706,9.488141e-04,2.337782e-03,1.029723e-03,1.897746e-03,1.838712e-01,6.083541e-04,...,4.391943e-03,3.473295e-04,7.473703e-04,0.001069,0.029292,1.883968e-04,3.713636e-03,0.000753,1.405053e-02,3.715561e-03
7,005ae625.wav,7.531418e-03,0.157932,0.011824,4.923249e-03,1.241653e-01,1.917691e-03,2.256909e-03,5.868747e-03,8.764904e-04,...,3.520455e-03,8.685963e-03,4.735946e-03,0.012409,0.000254,2.026742e-04,2.793622e-03,0.483905,1.552879e-02,1.266826e-03
8,007759c4.wav,1.319767e-03,0.170687,0.011814,2.058143e-03,1.026187e-01,1.862089e-04,1.739382e-03,8.556991e-02,1.220020e-03,...,2.449039e-03,1.003258e-03,3.250429e-04,0.012489,0.000411,2.254013e-04,4.396210e-03,0.006590,1.909893e-03,4.273223e-04
9,008afd93.wav,1.980706e-04,0.913784,0.005303,1.788423e-05,1.631423e-02,4.392582e-05,1.922413e-04,4.962025e-03,7.946226e-05,...,8.617763e-05,2.655731e-05,6.754878e-05,0.000153,0.000021,2.604312e-05,1.855054e-03,0.000481,1.076229e-02,2.411710e-05
